# Week 08. Multi Layer Perceptron

과제는 총 3개의 cell을 작성하도록 구성되어있습니다

1~3 문제는 영상과 실습자료에 나와있는 것들을 적절히 응용하시면 됩니다

주석의 경우 이미지, 테이블 등의 표현이 어려운 관계로 받지 않겠습니다

### 실습 00. Load dataset

Dataset covertype이며, 인공위성이 찍은 사진을 전처리하여 table data로 작성한 dataset입니다

train set으로 다양한 방법들을 교차검증하고, test set에서도 좋은 성능을 보이는지 확인해보겠습니다

train set과 test set을 불러와주세요

In [1]:
import pandas as pd
import numpy as np
import os

from plotly.subplots import make_subplots
from plotly import graph_objs as go


x_train=pd.read_csv('08_mlp_x_train.csv')
x_test=pd.read_csv('08_mlp_x_test.csv')
y_train=pd.read_csv('08_mlp_y_train.csv')
y_test=pd.read_csv('08_mlp_y_test.csv')

### 실습 01. K-Fold validation

train set으로부터 학습된 모델은 test set을 잘 예측해야 하는 목표를 갖고 있습니다

test을 잘 예측하기 위해 각 방법별로 교차검증을 시도하겠습니다

모든 train sample들에 평균적으로 가장 좋은 성능을 보이는 모델을 찾아보려합니다

다음 방법들을 5-Fold를 사용하여 교차검증해주세요

그리고 실제 test set에 대해 성능을 마지막으로 출력해주세요

사용할 분류 기법 : Gaussian Naive-bayes, k-Neighbors Nearest, Decision Tree, Random Forest, Logistics regression

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold

kf=KFold(shuffle=True)

test_index_set=set()

x=x_train
y=y_train

for train_index, test_index in kf.split(x_train, y_train):
    test_index_set.update(test_index)

In [3]:
import warnings
warnings.filterwarnings('ignore')

KNN_scores=[]
DT_scores=[]
Logistic_scores=[]
GNB_scores=[]
RF_scores=[]

KNN=KNeighborsClassifier()
DT=DecisionTreeClassifier()
logistic=LogisticRegression(max_iter=100)
GNB=GaussianNB()
RF=RandomForestClassifier()

for train_index, test_index in kf.split(x, y):
    x_train, x_test, y_train, y_test=\
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    KNN.fit(x_train, y_train.values.ravel())
    DT.fit(x_train, y_train.values.ravel())
    logistic.fit(x_train, y_train.values.ravel())
    GNB.fit(x_train, y_train.values.ravel())
    RF.fit(x_train, y_train.values.ravel())
    
    KNN_scores.append(KNN.score(x_test, y_test.values.ravel()))
    DT_scores.append(DT.score(x_test, y_test.values.ravel()))
    Logistic_scores.append(logistic.score(x_test, y_test.values.ravel()))
    GNB_scores.append(GNB.score(x_test, y_test.values.ravel()))
    RF_scores.append(RF.score(x_test, y_test.values.ravel()))

print(f'1. KNeighborsClassifier 평균 정확도 : {np.array(KNN_scores).mean()*100:.5}%, 정확도의 편차 : {np.array(KNN_scores).std():.3}')
print(f'2. DecisionTreeClassifier 평균 정확도 : {np.array(DT_scores).mean()*100:.5}%, 정확도의 편차 : {np.array(DT_scores).std():.3}')
print(f'3. LogisticRegression 평균 정확도 : {np.array(Logistic_scores).mean()*100:.5}%, 정확도의 편차 : {np.array(Logistic_scores).std():.3}')
print(f'4. GaussianNB 평균 정확도 : {np.array(GNB_scores).mean()*100:.5}%, 정확도의 편차 : {np.array(GNB_scores).std():.3}')
print(f'5. RandomForestClassifier 평균 정확도 : {np.array(RF_scores).mean()*100:.5}%, 정확도의 편차 : {np.array(RF_scores).std():.3}')

1. KNeighborsClassifier 평균 정확도 : 97.185%, 정확도의 편차 : 0.00163
2. DecisionTreeClassifier 평균 정확도 : 93.799%, 정확도의 편차 : 0.00174
3. LogisticRegression 평균 정확도 : 72.496%, 정확도의 편차 : 0.00349
4. GaussianNB 평균 정확도 : 71.505%, 정확도의 편차 : 0.00481
5. RandomForestClassifier 평균 정확도 : 95.739%, 정확도의 편차 : 0.00285


## 실습 02. Multi Layer Perceptron

MLP는 perceptron들을 다양한 구조로 엮어놓은 모델입니다

Perceptron의 layer 갯수와 각 층마다 존재하는 perceptorn의 갯수, perceptron의 output을 변형하는 함수등이 모델성능을 결정합니다

실습영상에서는 perceptron의 층수만 바꿔가며 간단하게 실험하였지만 과제로 다양한 MLP 모델을 시도해보겠습니다

In [4]:
from sklearn.neural_network import MLPClassifier

### 실습 02-1. Multi Layer Perceptron - The number of perceptron

perceptron의 갯수를 감소시켜가며 확인해보겠습니다

2개의 perceptron layer를 사용하면서 50개, 20개일때로 층별 perceptron의 숫자를 줄여보세요

그리고 2개의 모델을 5-fold로 교차검증하여 아래 결과를 제시해주세요

In [7]:
%%time

mlp1=MLPClassifier((50,50),
                 activation='identity',
                 max_iter=200,
                 n_iter_no_change=10,
                 )

mlp2=MLPClassifier((20,20),
                 activation='identity',
                 max_iter=200,
                 n_iter_no_change=10,
                 )

scores1=[]
scores2=[]

for train_index, test_index in kf.split(x, y):
    x_train, x_test, y_train, y_test=\
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    mlp1.fit(x_train, y_train.values.ravel())
    mlp2.fit(x_train, y_train.values.ravel())
    
    scores1.append(mlp1.score(x_test, y_test.values.ravel()))
    scores2.append(mlp2.score(x_test, y_test.values.ravel()))

print('1-1) 각 layer의 perceptron 수가 50인 모델의 score : ', scores1)
print(f'\n1-2) 각 layer의 perceptron 수가 50인 모델의 평균 정확도: {np.array(scores1).mean()*100:.5}%, 정확도의 편차 : {np.array(scores1).std():.3}\n\n')
print('2-1) 각 layer의 perceptron 수가 20인 모델의 score : ', scores2)
print(f'\n2-2) 각 layer의 perceptron 수가 20인 모델의 평균 정확도: {np.array(scores2).mean()*100:.5}%, 정확도의 편차 : {np.array(scores2).std():.3}')

1-1) 각 layer의 perceptron 수가 50인 모델의 score :  [0.7140275581658008, 0.695617799864468, 0.7431379193493731, 0.6859821529425054, 0.6502880379532362]

1-2) 각 layer의 perceptron 수가 50인 모델의 평균 정확도: 69.781%, 정확도의 편차 : 0.0307


2-1) 각 layer의 perceptron 수가 20인 모델의 score :  [0.6881635419019652, 0.7117686921165575, 0.7232576527730713, 0.7173839376482548, 0.7342143906020558]

2-2) 각 layer의 perceptron 수가 20인 모델의 평균 정확도: 71.496%, 정확도의 편차 : 0.0153
Wall time: 2min 34s


### 실습 02-1. Multi Layer Perceptron - Various output function for every perceptron

실습 영상에서 사용한 MLP는 각 perceptron output function으로 $f(x)=x$의 관계인 linear output을 사용하고 있습니다

이번에는 linear output을 바꿔 sigmoid를 사용해보겠습니다

MLP classifier의 activations parameter를 'identity'에서 'logistic' 또는 'tanh'로 바꿔주세요

그리고 실습 02-1.의 과정을 반복하여 결과를 제시해주세요

In [6]:
%%time

mlp1=MLPClassifier((100,),
                 activation='identity',
                 max_iter=200,
                 n_iter_no_change=10,
                 )

mlp2=MLPClassifier((100,),
                 activation='logistic',
                 max_iter=200,
                 n_iter_no_change=10,
                 )

mlp3=MLPClassifier((100,),
                 activation='tanh',
                 max_iter=200,
                 n_iter_no_change=10,
                 )

scores1=[]
scores2=[]
scores3=[]

for train_index, test_index in kf.split(x, y):
    x_train, x_test, y_train, y_test=\
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    mlp1.fit(x_train, y_train.values.ravel())
    mlp2.fit(x_train, y_train.values.ravel())
    mlp3.fit(x_train, y_train.values.ravel())
    
    scores1.append(mlp1.score(x_test, y_test.values.ravel()))
    scores2.append(mlp2.score(x_test, y_test.values.ravel()))
    scores3.append(mlp3.score(x_test, y_test.values.ravel()))

print('1-1) 기존 모델의 score : ', scores1)
print(f'\n1-2) 기존 모델의 평균 정확도: {np.array(scores1).mean()*100:.5}%, 정확도의 편차 : {np.array(scores1).std():.3}\n\n')
print('2-1) activations parameter=logistic인 모델의 score : ', scores2)
print(f'\n2-2) activations parameter=logistic인 모델의 평균 정확도: {np.array(scores2).mean()*100:.5}%, 정확도의 편차 : {np.array(scores2).std():.3}\n\n')
print('3-1) activations parameter=tanh인 모델의 score : ', scores3)
print(f'\n3-2) activations parameter=tanh인 모델의 평균 정확도: {np.array(scores3).mean()*100:.5}%, 정확도의 편차 : {np.array(scores3).std():.3}')

1-1) 기존 모델의 score :  [0.7394398012197877, 0.7304043370228146, 0.4962159719868971, 0.6708460408900938, 0.5866937761210889]

1-2) 기존 모델의 평균 정확도: 64.472%, 정확도의 편차 : 0.0921


2-1) activations parameter=logistic인 모델의 score :  [0.7977185452902643, 0.7680144567427152, 0.7922738054896645, 0.7941940585112391, 0.8079746978425393]

2-2) activations parameter=logistic인 모델의 평균 정확도: 79.204%, 정확도의 편차 : 0.0132


3-1) activations parameter=tanh인 모델의 score :  [0.7388750847074769, 0.7648520442737745, 0.7734101434541963, 0.7554501298994691, 0.7369253360442788]

3-2) activations parameter=tanh인 모델의 평균 정확도: 75.39%, 정확도의 편차 : 0.0143
Wall time: 4min 50s
